In [15]:
import pandas

# Считайте таблицу с признаками из файла features.csv
features = pandas.read_csv('./features.csv', index_col='match_id')

features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16,2449,0,4,1974,3,63


In [28]:
match_result_columns = [
    'duration',
    'radiant_win',
    'tower_status_radiant',
    'tower_status_dire',
    'barracks_status_radiant',
    'barracks_status_dire'
]

# Удалите признаки, связанные с итогами матча
match_result_data = features[match_result_columns]
y = match_result_data['radiant_win']

match_progress_data = features.drop(match_result_columns, axis=1)

In [19]:
progress_count = match_progress_data.count()

In [23]:
partial_columns_info = progress_count[progress_count != progress_count.max()]
print(partial_columns_info)

print(u"\nВопрос: Много ли пропусков в данных?")
print(u"Ответ: из {} признаков, частично заполнены {}".format(len(progress_count), len(partial_columns_info)))

print(u"\nВопрос: Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать" 
      u"обоснование, почему их значения могут быть пропущены.")
print(u"Ответ: Список названий признаков, имеющих пропуски:")
print(", ".join([key for key in partial_columns_info.keys()]))

print(u"""
Основание пропусков для признаков:
* first_blood_time: игровое время первой крови
* first_blood_team: команда, совершившая первую кровь (0 — Radiant, 1 — Dire)
* first_blood_player1: игрок, причастный к событию
* first_blood_player2: второй игрок, причастный к событию

Понятно из описания задачи: "Если событие "первая кровь" не успело произойти за первые 5 минут, 
то признаки принимают пропущенное значение"

Признаки:
* radiant_bottle_time
* radiant_courier_time
* radiant_flying_courier_time
* radiant_first_ward_time
* dire_bottle_time
* dire_courier_time
* dire_flying_courier_time
* dire_first_ward_time

описывают время наступления некоторого необязательного события игры. Судя по всему - в некоторых матчах данное
событие не наступило в первые 5 минут матча.
""")

first_blood_time               77677
first_blood_team               77677
first_blood_player1            77677
first_blood_player2            53243
radiant_bottle_time            81539
radiant_courier_time           96538
radiant_flying_courier_time    69751
radiant_first_ward_time        95394
dire_bottle_time               81087
dire_courier_time              96554
dire_flying_courier_time       71132
dire_first_ward_time           95404
dtype: int64

Вопрос: Много ли пропусков в данных?
Ответ: из 102 признаков, частично заполнены 12

Вопрос: Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них датьобоснование, почему их значения могут быть пропущены.
Ответ: Список названий признаков, имеющих пропуски:
first_blood_time, first_blood_team, first_blood_player1, first_blood_player2, radiant_bottle_time, radiant_courier_time, radiant_flying_courier_time, radiant_first_ward_time, dire_bottle_time, dire_courier_time, dire_flying_courier_time, dire_first_ward_tim

In [29]:
# Замените пропуски на нули с помощью функции fillna()
X = match_progress_data.fillna(0)

In [27]:
print(u"Вопрос: Какой столбец содержит целевую переменную? Запишите его название.")
print(u"Ответ: radiant_win (1, если победила команда Radiant, 0 — иначе)")

Вопрос: Какой столбец содержит целевую переменную? Запишите его название.
Ответ: radiant_win (1, если победила команда Radiant, 0 — иначе)


In [67]:
import time
import datetime
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.grid_search import GridSearchCV


def print_exec_time(func):
    def wrapper(*args, **kwargs):
        start_time = datetime.datetime.now()
        return_value = func(*args, **kwargs)
        end_time = datetime.datetime.now()
        print("Execution time of {} is {} (args: {}, kwargs: {})".format(
            func.__name__, end_time - start_time, args, kwargs)
        )
        
        return return_value

    return wrapper

# @print_exec_time
# def get_scores_by_n_estimators_grid(n_estimators):
    
#     grid = {'C': np.arange(0.1, 2.3, step=0.1)}
#     k_fold = KFold(len(y), n_folds=5, random_state=1, shuffle=True)
#     clf = GradientBoostingClassifier(
#         n_estimators=n_estimators,
#         random_state=1
#     )
#     gs = GridSearchCV(clf, grid, scoring='roc_auc', cv=k_fold, n_jobs=-1)
#     gs.fit(X, y)


#     clf = GradientBoostingClassifier(
#         n_estimators=n_estimators,
#         random_state=1,
#         learning_rate=learning_rate,
#         max_depth=max_depth,
#         subsample=subsample
#     )

#     k_fold = KFold(len(y), n_folds=5, random_state=1, shuffle=True)
#     scores = cross_val_score(clf, X, y, cv=k_fold, scoring='roc_auc', n_jobs=-1)
#     return scores.mean()

@print_exec_time
def get_scores_by_n_estimators(n_estimators, learning_rate=0.1, max_depth=3, subsample=1.0):
    clf = GradientBoostingClassifier(
        n_estimators=n_estimators,
        random_state=1,
        learning_rate=learning_rate,
        max_depth=max_depth,
        subsample=subsample
    )

    k_fold = KFold(len(y), n_folds=5, random_state=1, shuffle=True)
    scores = cross_val_score(clf, X, y, cv=k_fold, scoring='roc_auc', n_jobs=-1)
    return scores.mean()

In [69]:
all_scores = list()
for n_estimators in [200, 400, 700, 1000]:
    all_scores.append((n_estimators, get_scores_by_n_estimators(n_estimators=n_estimators, learning_rate=0.3, subsample=0.7)))

Execution time of get_scores_by_n_estimators is 0:20:11.380728 (args: (), kwargs: {'n_estimators': 200, 'subsample': 0.7, 'learning_rate': 0.3})
Execution time of get_scores_by_n_estimators is 0:40:53.841561 (args: (), kwargs: {'n_estimators': 400, 'subsample': 0.7, 'learning_rate': 0.3})
Execution time of get_scores_by_n_estimators is 1:12:03.034461 (args: (), kwargs: {'n_estimators': 700, 'subsample': 0.7, 'learning_rate': 0.3})
Execution time of get_scores_by_n_estimators is 1:43:07.855042 (args: (), kwargs: {'n_estimators': 1000, 'subsample': 0.7, 'learning_rate': 0.3})


In [70]:
all_scores

[(200, 0.71532383145178025),
 (400, 0.71471199024982923),
 (700, 0.70978495252079088),
 (1000, 0.70645424347752162)]